# PKDB-REST API
This notebook provides simple examples querying data from PK-DB
https://pk-db.com/api/v1/swagger/

In [13]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=2, compact=True, sort_dicts=False)
base_url = "http://0.0.0.0:8000/api/v1"

TypeError: __init__() got an unexpected keyword argument 'sort_dicts'

## Statistics
To get a basic overview of the content of PK-DB and the version use the `/statistics/` endpoint.

In [10]:
r = requests.get(base_url + '/statistics/')
print(r.json())

{'version': '0.9.2a3', 'study_count': 507, 'reference_count': 507, 'group_count': 1447, 'individual_count': 6212, 'intervention_count': 1391, 'output_count': 72068, 'output_calculated_count': 11651, 'timecourse_count': 3036, 'scatter_count': 37}


## Info nodes
Information in PK-DB is organized as info nodes. Meta-information is encoded in the form of the info nodes which for a given field encodes meta-data such as description, synonyms, annotations and database cross-references.

### Query single info node
To retrieve the available info nodes use the `/info_nodes/` endpoint.

In [11]:
r = requests.get(base_url + '/info_nodes/')
print(r.json())

{'current_page': 1, 'last_page': 52, 'next_page_url': 'http://0.0.0.0:8000/api/v1/info_nodes/?page=2', 'prev_page_url': None, 'data': {'count': 1030, 'data': [{'sid': '137mu', 'name': '137MU', 'label': '137MU', 'deprecated': False, 'ntype': 'substance', 'dtype': 'undefined', 'description': 'Metabolite of caffeine.', 'synonyms': ['1,3,7-MU'], 'parents': [], 'annotations': [], 'xrefs': [], 'measurement_type': None, 'substance': {'mass': None, 'charge': None, 'formula': None}}, {'sid': '137tmu', 'name': '137TMU', 'label': '137TMU', 'deprecated': False, 'ntype': 'substance', 'dtype': 'undefined', 'description': 'Metabolite of caffeine.', 'synonyms': [], 'parents': [], 'annotations': [], 'xrefs': [], 'measurement_type': None, 'substance': {'mass': None, 'charge': None, 'formula': None}}, {'sid': '13cco2', 'name': '13C-co2', 'label': '13C-carbon dioxide', 'deprecated': False, 'ntype': 'substance', 'dtype': 'undefined', 'description': '13C carbon dioxide is a (13)C-modified compound that is c

## Query data for single study

In [ ]:
r = requests.get(base_r)